In [1]:
from thermoclassifier.phases.net import PhaseClassifier
from thermoclassifier.dataset.dataset_creator import *
from torch.utils.data import DataLoader
import torch

In [8]:
measurement = 'C'
element = 'Ag'
t = False

dc = DatasetCreator(elements=[element], splits=(0.8, 0.2), validation=False, seq_len=1, measurement=measurement)
train_dataset, test_dataset, val_dataset = dc.get_datasets()

# Create the DataLoaders
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

# Network
net = PhaseClassifier(element=element, train=t, measurement=measurement)

New network had to be initialized as no network exists for this element and measurement. Training necessary!


In [6]:
# Training and testing routines

def train(net, train_loader, batch_size, seq_len, save=True):
    # Hyperparameters
    nr_epochs = 1000
    lr = 0.0025
    
    loss_func = nn.CrossEntropyLoss()
    optimizer = Adam(net.parameters(), lr=lr)
    
    losses = np.zeros([nr_epochs, ])
    
    # Save the net with the lowest training loss as the best net
    best_net = net
    best_loss = epoch(net, train_loader, loss_func, optimizer, batch_size, seq_len)
    
    for i in range(nr_epochs):
        losses[i] = epoch(net, train_loader, loss_func, optimizer, batch_size, seq_len)
        
        if losses[i] < best_loss:
            best_net = net
        
        if i % 10 == 0:
            print(losses[i])
            
    return best_net
        
def test(net, test_loader): 
    correct = 0
    incorrect = 0

    for d in test_loader:
        inp = d[:, :, :-1].squeeze(1)/1000
        prediction = net(inp.float()).argmax()
        target = d[:, :, -1][:, 0].long()
        if prediction == target:
            correct += 1
        else:
            incorrect += 1

    accuracy = correct/(correct + incorrect)
    print('Test accuracy: ', accuracy)
    
    return accuracy

In [7]:
test(net, test_loader)

Test accuracy:  0.6442244224422442


0.6442244224422442